In [0]:
import logging
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DateType, LongType 
from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils
import requests
from datetime import datetime
from pyspark.sql.functions import when, col, round, max

In [0]:
logger = logging.getLogger("DeBugLoggerGoldNintendo")

logger.setLevel(logging.INFO)

if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter('[%(levelname)s] %(asctime)s - %(message)s',
                                  '%Y-%m-%d %H:%M:%S')
    handler.setFormatter(formatter)
    logger.addHandler(handler)

In [0]:
def get_cdi_today():
    url = "https://brasilapi.com.br/api/taxas/v1/cdi"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        cdi = 1 + (data['valor'] / 100)
        return cdi
    except requests.exceptions.RequestException as e:
        print(f"Erro ao conectar à API da Brasil API: {e}")
        return None
    except (ValueError, IndexError) as e:
        print(f"Erro ao processar os dados da API: {e}")
        return None

cdi_data = get_cdi_today()

In [0]:
df_consoles = spark.read.table("nintendodatabricks_workspace.silver.consoles_silver")

In [0]:
df_price_desconto = df_consoles.withColumn("preco_desconto", round(col("preco") - (col("preco") * col("desconto")), 2))

In [0]:
df_price_parcelado = df_price_desconto.withColumn("preco_parcelado", round((col("valor_prestacao") * col("numero_parcelas")), 2))

In [0]:
df_save_desconto = df_price_parcelado.withColumn("save_desconto", round((col("preco") - col("preco_desconto")), 2))

In [0]:
df_save_parcelado = df_save_desconto.withColumn(
    "save_parcelado",
    when((col("preco") - col("preco_parcelado")) * cdi_data <= 0, 0)
    .otherwise(round(((col("preco") - col("preco_parcelado")) * cdi_data), 2))
)

In [0]:
df_pct_parcelado = df_save_parcelado.withColumn(
    "pct_preco_parcelado",
    round(100 -((col("preco_parcelado") / col("preco")) * 100), 2)
)

In [0]:
df_recomendacao = df_pct_parcelado.withColumn(
    "recomendacao",
    when(col("desconto") < col("pct_preco_parcelado"), "Comprar Parcelado")
    .when(col("desconto") > col("pct_preco_parcelado"), "Comprar a Vista Pix")
    .otherwise("Sem Vantagem Identificada")
).drop("preco_desconto", "preco_parcelado", "save_desconto", "save_parcelado", "pct_preco_parcelado")

In [0]:
display(df_recomendacao)

In [0]:
df_recomendacao.printSchema()